<a href="https://colab.research.google.com/github/AjeetSingh02/Notebooks/blob/master/recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
# !wget https://datasets.imdbws.com/title.basics.tsv.gz
# !wget https://datasets.imdbws.com/title.akas.tsv.gz
# !wget https://datasets.imdbws.com/title.crew.tsv.gz
# !wget https://datasets.imdbws.com/title.ratings.tsv.gz
# !wget https://datasets.imdbws.com/title.principals.tsv.gz

In [0]:
#Reading the datasets stored in tsv file format

title_basic_info = pd.read_csv("title.basics.tsv.gz", sep="\t")
title_more_info = pd.read_csv("title.akas.tsv.gz", sep="\t")
directors_writers = pd.read_csv("title.crew.tsv.gz", sep="\t")
ratings = pd.read_csv("title.ratings.tsv.gz", sep="\t")
crew_info_orig = pd.read_csv("title.principals.tsv.gz", sep="\t")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
title_basic_info.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [0]:
title_more_info.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
1,tt0000001,2,Καρμενσίτα,GR,\N,\N,\N,0
2,tt0000001,3,Карменсита,RU,\N,\N,\N,0
3,tt0000001,4,Carmencita,US,\N,\N,\N,0
4,tt0000001,5,Carmencita,\N,\N,original,\N,1


In [0]:
directors_writers.head()

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N


In [0]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.8,1497
1,tt0000002,6.3,181
2,tt0000003,6.6,1136
3,tt0000004,6.4,110
4,tt0000005,6.2,1837


In [0]:
crew_info_orig.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Herself""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N


**Will Start from title_basic_info, which is having some basic info about all the available titles. Then We will go to each dataset and extract important columns to make one combined dataset**

In [0]:
title_basic_info.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [0]:
title_basic_info["titleType"].value_counts()

tvEpisode       4050510
short            681705
movie            517334
video            229785
tvSeries         164262
tvMovie          126253
tvMiniSeries      25944
videoGame         23404
tvSpecial         17798
tvShort            9392
Name: titleType, dtype: int64

**Since There are a lot of types of videos available as "title" and we are interested in only movies. So we will extract only the rows corresponding to movies**

In [0]:
movies_df = title_basic_info[title_basic_info["titleType"] == "movie"]

In [0]:
movies_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
145,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,20,"Documentary,News,Sport"
332,tt0000335,movie,Soldiers of the Cross,Soldiers of the Cross,0,1900,\N,\N,"Biography,Drama"
499,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N
571,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Biography,Crime,Drama"


**Keeping only the rows which I think will be important in making recommendations.**

In [0]:
movies_df = movies_df[["tconst", "primaryTitle", "isAdult", "genres"]]

In [0]:
movies_df.head()

,tconst,primaryTitle,isAdult,genres
8,tt0000009,Miss Jerry,0,Romance
145,tt0000147,The Corbett-Fitzsimmons Fight,0,"Documentary,News,Sport"
332,tt0000335,Soldiers of the Cross,0,"Biography,Drama"
499,tt0000502,Bohemios,0,\N
571,tt0000574,The Story of the Kelly Gang,0,"Biography,Crime,Drama"


**Now let us see the title_more dataframe which has other important information about videos**

In [0]:
title_more_info.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
1,tt0000001,2,Καρμενσίτα,GR,\N,\N,\N,0
2,tt0000001,3,Карменсита,RU,\N,\N,\N,0
3,tt0000001,4,Carmencita,US,\N,\N,\N,0
4,tt0000001,5,Carmencita,\N,\N,original,\N,1


**First we will merge the two dataframes on the movie ID columns (tconst,titleID). Since the two columns have same meaning we will rename them to have same name.** 

In [0]:
title_more_info.rename(columns={"titleId":"tconst"}, inplace=True)

In [0]:
title_more_info.head(1)

,tconst,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0


In [0]:
title_full_df = movies_df.merge(title_more_info, on="tconst")

In [0]:
title_full_df.head()

,tconst,primaryTitle,isAdult,genres,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000009,Miss Jerry,0,Romance,1,Miss Jerry,\N,\N,original,\N,1
1,tt0000009,Miss Jerry,0,Romance,2,Miss Jerry,HU,\N,imdbDisplay,\N,0
2,tt0000009,Miss Jerry,0,Romance,3,Miss Jerry,US,\N,\N,\N,0
3,tt0000147,The Corbett-Fitzsimmons Fight,0,"Documentary,News,Sport",1,The Corbett-Fitzsimmons Fight,US,\N,\N,\N,0
4,tt0000147,The Corbett-Fitzsimmons Fight,0,"Documentary,News,Sport",2,Бой Корбетта и Фитцсиммонса,RU,\N,\N,\N,0


**Now keeping only the important columns and neglecting rest**

In [0]:
title_full_df.drop(columns = ["ordering","title","attributes","isOriginalTitle"], inplace=True)

In [0]:
title_full_df.head()

,tconst,primaryTitle,isAdult,genres,region,language,types
0,tt0000009,Miss Jerry,0,Romance,\N,\N,original
1,tt0000009,Miss Jerry,0,Romance,HU,\N,imdbDisplay
2,tt0000009,Miss Jerry,0,Romance,US,\N,\N
3,tt0000147,The Corbett-Fitzsimmons Fight,0,"Documentary,News,Sport",US,\N,\N
4,tt0000147,The Corbett-Fitzsimmons Fight,0,"Documentary,News,Sport",RU,\N,\N


In [0]:
#Removing any row will Null value
title_full_df.dropna(inplace=True)

In [0]:
title_full_df.isnull().any()

tconst          False
primaryTitle    False
isAdult         False
genres          False
region          False
language        False
types           False
dtype: bool

**Now we will remove all the rows where any value is "\N". We will do this because while making recommendations we cant decide any value for this. Though there are some other options also:**
1. Change the value to the mean of the whole column.
2. Give any random value between the highest and lowest value of the column.
3. Distribute all the \N value between all other non \N values.

**For now we will just remove all the rows and do the experiments later to check the impacts on the end result**

In [0]:
title_full_df = title_full_df[title_full_df["genres"] != "\\N"]
title_full_df = title_full_df[title_full_df["region"] != "\\N"]
title_full_df = title_full_df[title_full_df["language"] != "\\N"]
title_full_df = title_full_df[title_full_df["types"] != "\\N"]

In [0]:
title_full_df.head()

,tconst,primaryTitle,isAdult,genres,region,language,types
19,tt0000630,Hamlet,0,Drama,FI,sv,imdbDisplay
158,tt0001381,En rekrut fra 64,0,"Drama,War",FI,sv,imdbDisplay
193,tt0001603,Den farlige leg,0,Drama,FI,sv,imdbDisplay
211,tt0001692,The Great Moment,0,Drama,FI,sv,imdbDisplay
233,tt0001790,"Les Misérables, Part 1: Jean Valjean",0,Drama,XWW,en,imdbDisplay


**Now Let us see the next dataframe and extract important information.**

In [0]:
crew_info_orig.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Herself""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N


In [0]:
crew_info_orig.category.value_counts()

actor                  7915976
actress                5776153
self                   5704821
writer                 4360988
director               3790419
producer               2014147
composer               1228152
cinematographer        1197061
editor                 1112474
production_designer     266600
archive_footage         193453
archive_sound             1991
Name: category, dtype: int64

**We will take only the rows corresponding to Director, Actor or Actress as they are more influential**

In [0]:
crew_info = crew_info_orig[(crew_info_orig.category == "actor") | (crew_info_orig.category == "actress") | (crew_info_orig.category == "director")]

In [0]:
crew_info.head()

,tconst,ordering,nconst,category,job,characters
1,tt0000001,2,nm0005690,director,\N,\N
3,tt0000002,1,nm0721526,director,\N,\N
5,tt0000003,1,nm0721526,director,\N,\N
9,tt0000004,1,nm0721526,director,\N,\N
11,tt0000005,1,nm0443482,actor,\N,"[""Blacksmith""]"


In [0]:
crew_info.ordering.value_counts()

1     3495289
2     3095175
3     2789466
4     2499487
5     2218562
6      741227
9      688621
10     681570
8      651193
7      621958
Name: ordering, dtype: int64

**There are multiple actors, actresses and directors of a movie. We will take only 3 top ranked. This information is given by "ordering" column. Numbers 1-10 tells what priority that person has in the movie. Then we will drop the ordering column as it has served its purpose**

In [0]:
crew_info = crew_info[crew_info.ordering.isin([1,2,3])]
crew_info.drop(columns=["ordering"], inplace=True)

In [0]:
crew_info.head()

,tconst,nconst,category,job,characters
1,tt0000001,nm0005690,director,\N,\N
3,tt0000002,nm0721526,director,\N,\N
5,tt0000003,nm0721526,director,\N,\N
9,tt0000004,nm0721526,director,\N,\N
11,tt0000005,nm0443482,actor,\N,"[""Blacksmith""]"


**The columns "Job" and "characters" gives information about what kind of director a "director" is and what roles did an actor/actress played. So we will remove those columns as they have no value for recommendation for now. We will later see if they can be any use**

In [0]:
crew_info.drop(columns=["job","characters"], inplace=True)

In [0]:
crew_info.head()

,tconst,nconst,category
1,tt0000001,nm0005690,director
3,tt0000002,nm0721526,director
5,tt0000003,nm0721526,director
9,tt0000004,nm0721526,director
11,tt0000005,nm0443482,actor


**Now we will make separate columns for Actors, Actress and Directors. There can be multiple approaches for this, but the one that I have used is that 1st create separate datasets for all three and then merge with original and then remove the extra category column**

In [0]:
directors_df = crew_info[crew_info.category=="director"]
actors_df = crew_info[crew_info.category == "actor"]
actress_df = crew_info[crew_info.category == "actress"]

In [0]:
directors_df.drop(columns=["category"], inplace=True)
actors_df.drop(columns=["category"], inplace=True)
actress_df.drop(columns=["category"], inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [0]:
directors_df.head()

,tconst,nconst
1,tt0000001,nm0005690
3,tt0000002,nm0721526
5,tt0000003,nm0721526
9,tt0000004,nm0721526
13,tt0000005,nm0005690


In [0]:
crew_info.head()

,tconst,nconst,category
1,tt0000001,nm0005690,director
3,tt0000002,nm0721526,director
5,tt0000003,nm0721526,director
9,tt0000004,nm0721526,director
11,tt0000005,nm0443482,actor


**As we can see the crew_info has "nconst" column which is already in respective category datasets. Also the "category" column is redundant. SO we will remove these two columns and then merge the three category datasets with the crew_info_updated dataset.
Also there will be multiple copies of same "tconst" in the crew_info as there were separate rows for actor, director and actress. But now since we have separate columns for the same we will remove duplicate records**

In [0]:
#Dropping the columns
crew_info.drop(columns=["nconst","category"], inplace=True)

In [0]:
#Merging the Director dataframe
crew_info = crew_info.merge(directors_df, on="tconst")
crew_info.rename(columns = {"nconst":"director"}, inplace=True)

In [0]:
#Merging the Actor dataframe
crew_info = crew_info.merge(actors_df, on="tconst")
crew_info.rename(columns = {"nconst":"actor"}, inplace=True)

In [0]:
#Merging the Actress dataframe
crew_info = crew_info.merge(actress_df, on="tconst")
crew_info.rename(columns = {"nconst":"actress"}, inplace=True)

In [0]:
#Dropping the duplicates
crew_info.drop_duplicates(inplace=True)

In [0]:
crew_info.head()

,tconst,director,actor,actress
0,tt0000017,nm1587194,nm3691272,nm3692829
3,tt0000172,nm0808310,nm0338379,nm0809419
6,tt0000308,nm0808310,nm2940072,nm2940350
9,tt0000313,nm0808310,nm0338379,nm0809419
12,tt0000442,nm0253298,nm1854449,nm0622273


In [0]:
title_full_df.head()

,tconst,primaryTitle,isAdult,genres,region,language,types
19,tt0000630,Hamlet,0,Drama,FI,sv,imdbDisplay
158,tt0001381,En rekrut fra 64,0,"Drama,War",FI,sv,imdbDisplay
193,tt0001603,Den farlige leg,0,Drama,FI,sv,imdbDisplay
211,tt0001692,The Great Moment,0,Drama,FI,sv,imdbDisplay
233,tt0001790,"Les Misérables, Part 1: Jean Valjean",0,Drama,XWW,en,imdbDisplay


**Now we will merge crew_info dataframe with our earlier title_full_df dataframe. So that we now have combined information of both the dataframes**

In [0]:
title_full_df = title_full_df.merge(crew_info, on="tconst")

In [0]:
title_full_df.head()

,tconst,primaryTitle,isAdult,genres,region,language,types,director,actor,actress
0,tt0018353,Sabishiki ranbomono,0,Drama,XWW,en,alternative,nm0331482,nm1263070,nm1846969
1,tt0045040,A Treasure on Bird Island,0,Animation,XEU,en,imdbDisplay,nm0954724,nm1227995,nm1785269
2,tt0045040,A Treasure on Bird Island,0,Animation,XWW,en,imdbDisplay,nm0954724,nm1227995,nm1785269
3,tt0050490,"Heaven Knows, Mr. Allison",0,"Adventure,Drama,War",CA,en,imdbDisplay,nm0001379,nm0000053,nm0000039
4,tt0076652,Safrana or Freedom of Speech,0,Drama,XWW,en,imdbDisplay,nm0812406,nm0393124,nm1429466


**Now we have one more dataset remaining. That is related to ratings. Let us see that also**

In [0]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.8,1497
1,tt0000002,6.3,181
2,tt0000003,6.6,1136
3,tt0000004,6.4,110
4,tt0000005,6.2,1837


In [0]:
len(ratings)

931372

**Merging "ratings" dataframe with "title_full_df"**

In [0]:
movie_final_df = title_full_df.merge(ratings, on="tconst")

In [0]:
movie_final_df.head()

,tconst,primaryTitle,isAdult,genres,region,language,types,director,actor,actress,averageRating,numVotes
0,tt0045040,A Treasure on Bird Island,0,Animation,XEU,en,imdbDisplay,nm0954724,nm1227995,nm1785269,6.6,62
1,tt0045040,A Treasure on Bird Island,0,Animation,XWW,en,imdbDisplay,nm0954724,nm1227995,nm1785269,6.6,62
2,tt0050490,"Heaven Knows, Mr. Allison",0,"Adventure,Drama,War",CA,en,imdbDisplay,nm0001379,nm0000053,nm0000039,7.4,7206
3,tt0078196,Made in Iran,0,"Crime,Drama",XWW,en,imdbDisplay,nm0618881,nm0705282,nm0074038,7.0,47
4,tt0108615,Red Beads,0,Drama,XWW,en,imdbDisplay,nm0372087,nm0422629,nm0443752,6.1,10


**The "region", "language", "types", "director", "actor","actress" columns can have different values for same movie. So What we will do is join all the values of a particular column corresponding to a movie together. Before that we will store all the unique values for further use.** 

In [0]:
regions_unique = movie_final_df.genres.unique()
languages_unique = movie_final_df.language.unique()
types_unique = movie_final_df.types.unique()
directors_unique = movie_final_df.director.unique()
actors_unique = movie_final_df.actor.unique()
actresses_unique = movie_final_df.actress.unique()

**we will do the grouping and joining of individual columns separately in 4 steps.**
1. Make a temp dataframe with just "tconst" and the column whose values we want to join.
2. Drop the column from movie_final_df dataframe.
3. Group by "tconst" and join the all the values corresponding to this "tconst" by comma(,).
4.Drop duplicates from original_dataset
5. Merge this temperory dataframe with the original dataframe

In [0]:
movie_final_df.head()

,tconst,primaryTitle,isAdult,genres,region,language,types,director,actor,actress,averageRating,numVotes
0,tt0045040,A Treasure on Bird Island,0,Animation,XEU,en,imdbDisplay,nm0954724,nm1227995,nm1785269,6.6,62
1,tt0045040,A Treasure on Bird Island,0,Animation,XWW,en,imdbDisplay,nm0954724,nm1227995,nm1785269,6.6,62
2,tt0050490,"Heaven Knows, Mr. Allison",0,"Adventure,Drama,War",CA,en,imdbDisplay,nm0001379,nm0000053,nm0000039,7.4,7206
3,tt0078196,Made in Iran,0,"Crime,Drama",XWW,en,imdbDisplay,nm0618881,nm0705282,nm0074038,7.0,47
4,tt0108615,Red Beads,0,Drama,XWW,en,imdbDisplay,nm0372087,nm0422629,nm0443752,6.1,10


In [0]:
len(movie_final_df)

157

In [0]:
temp_df = movie_final_df[["tconst", "region"]]
movie_final_df.drop(columns=["region"], inplace=True)
temp_df = temp_df.groupby("tconst").agg({"region":", ".join})
movie_final_df.drop_duplicates(inplace=True)
movie_final_df = movie_final_df.merge(temp_df, on="tconst")

In [0]:
temp_df = movie_final_df[["tconst", "language"]]
movie_final_df.drop(columns=["language"], inplace=True)
temp_df = temp_df.groupby("tconst").agg({"language":", ".join})
movie_final_df.drop_duplicates(inplace=True)
movie_final_df = movie_final_df.merge(temp_df, on="tconst")

In [0]:
temp_df = movie_final_df[["tconst", "types"]]
movie_final_df.drop(columns=["types"], inplace=True)

temp_df = temp_df.groupby("tconst").agg({"types":", ".join})
movie_final_df.drop_duplicates(inplace=True)
movie_final_df = movie_final_df.merge(temp_df, on="tconst")

In [0]:
temp_df = movie_final_df[["tconst", "director"]]
movie_final_df.drop(columns=["director"], inplace=True)

temp_df = temp_df.groupby("tconst").agg({"director":", ".join})
movie_final_df.drop_duplicates(inplace=True)
movie_final_df = movie_final_df.merge(temp_df, on="tconst")

In [0]:
temp_df = movie_final_df[["tconst", "actor"]]
movie_final_df.drop(columns=["actor"], inplace=True)

temp_df = temp_df.groupby("tconst").agg({"actor":", ".join})
movie_final_df.drop_duplicates(inplace=True)
movie_final_df = movie_final_df.merge(temp_df, on="tconst")

In [0]:
temp_df = movie_final_df[["tconst", "actress"]]
movie_final_df.drop(columns=["actress"], inplace=True)

temp_df = temp_df.groupby("tconst").agg({"actress":", ".join})
movie_final_df.drop_duplicates(inplace=True)
movie_final_df = movie_final_df.merge(temp_df, on="tconst")

In [0]:
#dropping numVotes column for now
movie_final_df.drop(columns='numVotes', inplace=True)

In [0]:
movie_final_df.head()

,tconst,primaryTitle,isAdult,genres,averageRating,region,language,types,director,actor,actress
0,tt0045040,A Treasure on Bird Island,0,Animation,6.6,"XEU, XWW",en,imdbDisplay,nm0954724,nm1227995,nm1785269
1,tt0050490,"Heaven Knows, Mr. Allison",0,"Adventure,Drama,War",7.4,CA,en,imdbDisplay,nm0001379,nm0000053,nm0000039
2,tt0078196,Made in Iran,0,"Crime,Drama",7.0,XWW,en,imdbDisplay,nm0618881,nm0705282,nm0074038
3,tt0108615,Red Beads,0,Drama,6.1,XWW,en,imdbDisplay,nm0372087,nm0422629,nm0443752
4,tt0119711,Mother and Son,0,Drama,7.5,"XWW, JP",en,imdbDisplay,nm0812546,nm0025651,nm0315407


In [0]:
len(movie_final_df)

157

In [0]:
genres_unique = movie_final_df.genres.unique()

In [0]:
genres = []
for genre_string in genres_unique:
  genre_list = genre_string.split(",")
  for genre in genre_list:
    if genre not in genres:
      genres.append(genre)

In [0]:
size = len(movie_final_df)

In [0]:
temp_matrix = np.zeros([size,len(genres)])

In [0]:
for i in range(size):
  genre_string = movie_final_df.genres[i]
  genre_list = genre_string.split(",")
  for genre in genre_list:
    ind = genres.index(genre)
    temp_matrix[i][ind] = 1


In [0]:
temp_df = pd.DataFrame(temp_matrix, columns = genres)

In [0]:
len(temp_df)

157